In [51]:
#Importing Dependencies
%matplotlib inline
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import plotly.express as px
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn import utils
from functools import reduce
from sklearn.cluster import KMeans
import psycopg2
from sqlalchemy import create_engine
from config import DB_HOST, DB_USER, DB_PASS, DB_PORT

In [52]:
#Creating engine to access online databse
engine = create_engine(f'postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/us_emissions_db')

In [53]:
#reading merged table and year averages table
merged_table = pd.read_sql_table('gdp_pop_sector_emissions', con=engine)
year_avg = pd.read_sql_table('year_averages', con=engine)
year_avg.columns = ['year', 'avgpop', 'avggdp', 'avgghg']

In [54]:
#displaying merged table and year table
display(merged_table.head())
display(year_avg.head())

,Country,State,Year,GDP,Population,Sector,allghg,CH4,CO2,N2O,F-Gas
0,US,AL,1997,3.724787e+10,4367935,Agriculture,6.889241,3.392917,0.016045,4.028508,1.128290
1,US,AL,1998,3.936806e+10,4404701,Agriculture,6.954451,3.137611,0.016269,3.751630,1.290703
2,US,AL,1999,4.151366e+10,4430141,Agriculture,6.560038,3.074048,0.013043,3.880403,1.376542
3,US,AL,2000,4.258337e+10,4452173,Agriculture,6.298967,2.959365,0.002420,3.600673,1.521127
4,US,AL,2001,4.334858e+10,4467634,Agriculture,6.374233,2.899730,0.007422,3.399237,1.600098


,year,avgpop,avggdp,avgghg
0,1997,5.346018e+06,6.336021e+10,19.723300
1,1998,5.408904e+06,6.529091e+10,19.659627
2,1999,5.471376e+06,6.939920e+10,19.800945
3,2000,5.532596e+06,7.215153e+10,20.297083
4,2001,5.587627e+06,7.477159e+10,19.961042


# Random Forest Regressor Model

In [55]:
#Creating target and features for Random Forest Regressor Model to be used on the Avg Table
X = year_avg[['avgpop','avggdp']]
y = year_avg['avgghg']

In [56]:
#Scaling the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [57]:
#Train and test data
clf = RandomForestRegressor(random_state=1, n_estimators=50).fit(X_train, y_train)

print(f'Training Score: {clf.score(X_train, y_train)}')
print(f'Testing Score: {clf.score(X_test, y_test)}')

Training Score: 0.963617877979699
Testing Score: 0.6901170803933703


In [119]:
#Create function to predict ghg based on Population & GDP per sector & state
def rfr_model(state,sector):
    model_data = merged_table.loc[(merged_table['State']==state) & (merged_table['Sector']==sector)]
    X = model_data[['Population','GDP']]
    y = model_data['allghg']
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
    scaler = StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    clf = RandomForestRegressor(random_state=1, n_estimators=50).fit(X_train, y_train)
    print(f'Training Score: {clf.score(X_train, y_train)}')
    print(f'Testing Score: {clf.score(X_test, y_test)}')

In [121]:
rfr_model('NJ', 'Agriculture')

Training Score: 0.9493439960319076
Testing Score: 0.6027889163029333


C:\ProgramData\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name']. An error will be raised in 1.2.
  FutureWarning,
C:\ProgramData\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name']. An error will be raised in 1.2.
  FutureWarning,
C:\ProgramData\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name']. An error will be raised in 1.2.
  FutureWarning,
C:\ProgramData\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['quoted_name']. An error will be raised in 1.